In [1]:
import os
os.chdir("../")

In [2]:
import numpy as np
import torch

In [3]:
from storage.har_datasets import REALDISPDataset, sts_medoids, split_by_test_subject
from s3ts.api.dms.har_datasets import LDFDataset, DFDataset
from storage.label_mappings import *
from s3ts.api.nets.methods import create_model_from_DM, train_model, test_model
from torchvision.transforms import Normalize

In [4]:
ds = REALDISPDataset("./datasets/REALDISP/", sensor_position=["LLA", "BACK"], sensor=["acc"], mode=["ideal"], wsize=48, normalize=True, label_mapping=None)
print(len(ds))

2579760


In [5]:
if not os.path.exists("./datasets/REALDISP/meds.npz"):
    meds = sts_medoids(ds, n=500)
    with open("./datasets/REALDISP/meds.npz", "wb") as f:
        np.save(f, meds)
else:
    meds = np.load("./datasets/REALDISP/meds.npz")

dfds = DFDataset(ds, patterns=meds, w=0.1, dm_transform=None, ram=False)

Loading cached dissimilarity frames if available...


In [6]:
DM = []

np.random.seed(42)
for i in np.random.choice(np.arange(len(dfds)), 500):
    dm, _, _ = dfds[i]
    DM.append(dm)

DM = torch.stack(DM)

dm_transform = Normalize(mean=DM.mean(dim=[0, 2, 3]), std=DM.std(dim=[0, 2, 3]))
dfds.dm_transform = dm_transform

In [7]:
data_split = split_by_test_subject(ds, 16)

dm = LDFDataset(dfds, data_split=data_split, batch_size=128, random_seed=42, num_workers=8)

In [8]:
print(len(dm.ds_train))
print(len(dm.ds_val))
print(len(dm.ds_test))

2374310
205450
205450


In [9]:
model = create_model_from_DM(dm, name=None, 
        dsrc="img", arch="cnn", task="cls")

Input shape:  torch.Size([1, 34, 48, 48])
Latent shape:  torch.Size([1, 40, 3, 48])


In [ ]:
model, data = train_model(dm, model, max_epochs=2)
print(data)